# 12.3 실습: HNSW 인덱스의 핵심 파라미터 이해하기

- HNSW의 성능 지표를 제어하는 파라미터
    - 최소 연결 수 (m)
    - 색인 과정에서 가장 가까운 M개를 선택하기 위해 저장하는 후보의 수 (ef_construction)
    - 검색 과정에서 가장 가까운 K개를 선택할 때 저장하는 후보의 수 (ef_search)

## 12.3.1 파라미터 m 이해하기

- 최소 연결 수 (m)
    - 간선이 많을 수록 그래프가 더 촘촘하게 연결
    - 재현율 ↑ / 메모리 효율, 색인 속도, 검색 속도 ↓

In [ ]:
# 예제 12.2

import psutil

def get_memory_usage_mb():
    process = psutil.Process()
    memory_info = process.memory_info()
    return memory_info.rss / (1024 * 1024)

In [ ]:
# 예제 12.2

import time
import faiss
from faiss.contrib.datasets import DatasetSIFT1M

ds = DatasetSIFT1M()

xq = ds.get_queries()
xb = ds.get_database()
gt = ds.get_groundtruth()

In [ ]:
# 예제 12.3

k=1
d = xq.shape[1]
nq = 1000
xq = xq[:nq]

for i in range(1, 10, 2):
    start_memory = get_memory_usage_mb()
    start_indexing = time.time()
    index = faiss.IndexFlatL2(d)
    index.add(xb[:(i+1)*100000])
    end_indexing = time.time()
    end_memory = get_memory_usage_mb()

    t0 = time.time()
    D, I = index.search(xq, k)
    t1 = time.time()
    print(f"데이터 {(i+1)*100000}개:")
    print(f"색인: {(end_indexing - start_indexing) * 1000 :.3f} ms ({end_memory - start_memory:.3f} MB) 검색: {(t1 - t0) * 1000 / nq :.3f} ms")

In [ ]:
from IPython.display import display, clear_output, Markdown

def printMD(x):
    clear_output()
    display(Markdown(x))

In [ ]:
# 예제 12.4

import numpy as np

k=1
d = xq.shape[1]
nq = 1000
xq = xq[:nq]

table = """
| M | 메모리 사용량(MB) | 색인 시간(s) | 검색 시간(ms) | 재현율 |
| --- | --- | --- | --- | --- |
"""
printMD(table)

for m in [8, 16, 32, 64]:
    index = faiss.IndexHNSWFlat(d, m)
    time.sleep(3)
    start_memory = get_memory_usage_mb()
    start_index = time.time()
    index.add(xb)
    end_memory = get_memory_usage_mb()
    end_index = time.time()
    table = table + f"| {m} | {end_memory - start_memory:.5f} | {end_index - start_index:.5f}"

    t0 = time.time()
    D, I = index.search(xq, k)
    t1 = time.time()

    recall_at_1 = np.equal(I, gt[:nq, :1]).sum() / float(nq)

    table = table + f"| {(t1 - t0) * 1000.0 / nq:.3f} | {recall_at_1:.3f} |" + "\n"
    printMD(table)

## 12.3.2 파라미터 ef_construction 이해하기

- 색인 과정에서 가장 가까운 M개를 선택하기 위해 저장하는 후보의 수 (ef_construction)
    - ef_construction이 크면 더 많은 후보를 탐색하기 때문에 실제로 추가한 벡터와 가장 가까운 벡터를 선택할 가능성이 높아진다.
    - 하지만 많은 후보를 탐색하기 때문에 색인 시간이 증가한다.
    - 재현율 ↑ / 색인 속도 ↓ / 메모리 효율, 검색 속도 영향 X

In [ ]:
# 예제 12.5

k=1
d = xq.shape[1]
nq = 1000
xq = xq[:nq]

table = """
| ef_construction | 메모리 사용량(MB) | 색인 시간(s) | 검색 시간(ms) | 재현율 |
| --- | --- | --- | --- | --- |
"""
printMD(table)

for ef_construction in [40, 80, 160, 320]:
    index = faiss.IndexHNSWFlat(d, 32)
    index.hnsw.efConstruction = ef_construction
    time.sleep(3)
    start_memory = get_memory_usage_mb()
    start_index = time.time()
    index.add(xb)
    end_memory = get_memory_usage_mb()
    end_index = time.time()
    table = table + f"| {ef_construction} | {end_memory - start_memory:.5f} | {end_index - start_index:.5f}"

    t0 = time.time()
    D, I = index.search(xq, k)
    t1 = time.time()

    recall_at_1 = np.equal(I, gt[:nq, :1]).sum() / float(nq)
    table = table + f"| {(t1 - t0) * 1000.0 / nq:.3f} | {recall_at_1:.3f}" + "\n"
    printMD(table)

## 12.3.3 파라미터 ef_search 이해하기

- 검색 과정에서 가장 가까운 K개를 선택할 때 저장하는 후보의 수 (ef_search)
    - 재현율 ↑ / 검색 속도 ↓ / 메모리 효율, 색인 속도 영향 X

In [ ]:
# 예제 12.6

table = """
| ef_search | 검색 시간(ms) | 재현율 |
| --- | --- | --- |
"""
printMD(table)

for ef_search in [16, 32, 64, 128]:
    index.hnsw.efSearch = ef_search
    t0 = time.time()
    D, I = index.search(xq, k)
    t1 = time.time()

    recall_at_1 = np.equal(I, gt[:nq, :1]).sum() / float(nq)
    table = table + f"| {ef_search} | {(t1 - t0) * 1000.0 / nq:.3f} | {recall_at_1:.3f} |" + "\n"
    printMD(table)